In [ ]:
#!pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
print(os.environ["GOOGLE_API_KEY"])

In [ ]:
#!pip install langchain==0.1.*
#!pip install langchain-community==0.0.*
#!pip install langchain-openai==0.1.*
#!pip install openai
#!pip install weaviate-client==3.26.2

In [ ]:
#!pip install langchain_experimental -q

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
with open('La-Republilca-Platon.txt', encoding='utf-8') as f:
    repubica = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=60,
    length_function=len
)

chunks = text_splitter.create_documents([repubica])

#print(chunks)
# Para ver cuántos fragmentos se tiene, se imprime el tamaño de los chunk.
print(f'Hay {len(chunks)} fragmentos')

In [ ]:
# Se imprime una muestra del contenido
print(chunks[0].page_content)

## Calcular la cantidad de tokens de un texto:

In [ ]:
import tiktoken
def print_embedding_tokens(texts):
    enc = tiktoken.encoding_for_model('text-embedding-3-large')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    return total_tokens

total_tokens = print_embedding_tokens(chunks)
print(f'Total Tokens: {total_tokens}')

## Crear Embedding

In [ ]:
# No se usa por costos de uso del modelo
# from langchain_openai import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings(model='text-embedding-3-large', dimensions=3072)
# vector = embeddings.embed_query(chunks[0].page_content)
# vector

In [ ]:
#!pip install google.genai

In [ ]:
from google import genai
from google.genai import types

client = genai.Client()
text = chunks[0].page_content
embeddings = client.models.embed_content(
    model="text-embedding-004",
    contents=text,
    config=types.EmbedContentConfig(output_dimensionality=92),
)
print(embeddings.embeddings)

## Insertar el vector en weaviate

In [ ]:
#!pip uninstall weaviate-client -y
#!pip uninstall weaviate-client -y
#!pip install weaviate-client==3.26.2
#!pip install weaviate-client==3.26.2
#!pip install sentence-transformers
# !pip install --upgrade pip setuptools wheel
#!pip install numpy
#!pip install langchain-weaviate
#!pip install sentence-transformers faiss-cpu
!pip install --upgrade faiss-cpu numpy                                                                                                                                               

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# 1. Modelo de embeddings local
model = SentenceTransformer('all-MiniLM-L6-v2')

# 2. Tus documentos (pueden venir de cualquier fuente)
documentos = [
    "LangChain es una biblioteca para construir aplicaciones LLM.",
    "FAISS es una biblioteca para búsquedas de similitud usando vectores.",
    "Transformers son modelos de deep learning como BERT o GPT.",
    "La inteligencia artificial está transformando muchas industrias.",
    "Python es un lenguaje de programación muy popular."
]

# 3. Crear embeddings
embeddings = model.encode(documentos)

# 4. Crear índice FAISS (índice de similitud)
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

# 5. Consulta
consulta = "¿Qué es LangChain?"
embedding_consulta = model.encode([consulta])

# 6. Búsqueda en el índice
k = 2  # top-k resultados
distancias, indices = index.search(np.array(embedding_consulta), k)

print("\n🔍 Resultados para la consulta:", consulta)
for i, idx in enumerate(indices[0]):
    print(f"{i+1}. {documentos[idx]} (Distancia: {distancias[0][i]:.4f})")


In [ ]:
!pip install pymupdf langchain

In [ ]:
import fitz  # PyMuPDF
from langchain.schema import Document

def cargar_pdf_a_documentos(path_pdf):
    documentos = []
    with fitz.open(path_pdf) as doc:
        for i, pagina in enumerate(doc):
            texto = pagina.get_text()
            if texto.strip():  # Evita páginas en blanco
                documentos.append(Document(page_content=texto, metadata={"page": i + 1}))
    return documentos


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# 1. Cargar el PDF
docs = cargar_pdf_a_documentos("C:/Users/wilpa/OneDrive/Escritorio/Libros/Generative-AI-with-LangChain-2024-Original.pdf")

# 2. Embeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# 3. Crear vectorstore con FAISS
vectorstore = FAISS.from_documents(docs, embedding_model)

# 4. Guardar localmente si quieres reutilizar
vectorstore.save_local("vectorstore_pdfs")


In [ ]:
# Recargar (si ya lo habías guardado)
# vectorstore = FAISS.load_local("vectorstore_pdfs", embedding_model)

query = "¿Cuál es el propósito del documento?"
resultados = vectorstore.similarity_search(query, k=3)

for i, doc in enumerate(resultados):
    print(f"\nDocumento {i+1}: Página {doc.metadata['page']}")
    print(doc.page_content[:500])  # Mostrar primeros 500 caracteres


In [ ]:
#!pip install langchain-google-genai
#!pip install --upgrade protobuf
#!python -c "import google.protobuf; print(google.protobuf.__version__)"
#!pip install --upgrade protobuf
#!pip install --upgrade protobuf
!pip install google-cloud-core google-cloud-storage google-cloud-bigquery google-api-core

In [ ]:
import fitz  # PyMuPDF
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

# 1. Función para leer PDF y cargar documentos
def cargar_pdf_a_documentos(path_pdf):
    documentos = []
    with fitz.open(path_pdf) as doc:
        for i, pagina in enumerate(doc):
            texto = pagina.get_text()
            if texto.strip():
                documentos.append(Document(page_content=texto, metadata={"page": i + 1}))
    return documentos

# 2. Carga documentos desde PDF
docs = cargar_pdf_a_documentos("C:/Users/wilpa/OneDrive/Escritorio/Libros/Cloud-Data-Warehousing-For-Dummies-3rd-Edition.pdf")

# 3. Crear embeddings (OpenAI en este ejemplo)
#embedding_model = OpenAIEmbeddings()
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# 4. Crear vectorstore FAISS con los documentos embebidos
vectorstore = FAISS.from_documents(docs, embedding_model)

# 5. Crear modelo LLM OpenAI (puedes usar otras opciones también)
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    #model="gemini-1.5-pro",
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)
#llm = OpenAI(temperature=0)

# 6. Crear chain RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())

# 7. Preguntar
query = "¿Cuál es el propósito del documento?"
respuesta = qa_chain.run(query)

print("Respuesta:", respuesta)


In [ ]:
!pip install --upgrade protobuf==6.31.0

In [ ]:
#!pip install langchain

In [ ]:
#!pip install langchain-cohere

In [ ]:
#!pip install langchain-huggingface
#!pip install huggingface_hub transformers sentence-transformers

In [ ]:
#!pip install langchain-chroma

In [ ]:
#!pip install langchain-community faiss-cpu  

In [ ]:
#!pip install langchain-pinecone

In [ ]:
#!pip install langchain-weaviate

In [ ]:
#!pip install pypdf

In [ ]:
#!pip install beautifulsoup4

In [ ]:
#!pip install unstructured

In [ ]:
#!pip install google-search-results

In [ ]:
#!pip install wikipedia

In [ ]:
#pip install langchain[all]

In [ ]:
#!pip install weaviate-client>=4.0.0